# Properties
<br><img width=600 src="Images/Property.png" />

Nachdem wir im letzten Abschnitt bereits die Anfänge eines Schutzkonzepts für unsere Klassen besprochen haben, möchten wir jetzt die effektivste Art zeigen, wie man dies umsetzt. Wir hatten getter- und setter-Methoden gezeigt und den Schutz von Attributen, indem man sie privat macht. Diese Art mit den Zugriffsmethoden und privaten Attributen zu arbeiten, ist in vielen Programmiersprachen gang und gäbe. Auch in Python ist dies weit verbreitet, manche entwickeln regelrecht den Reflex, automatisch für jedes Attribut einen Getter und Setter zu benutzen und das Attribut zu privatisieren, es gibt sogar IDEs, die einen fast zwingen, dies zu tun. Wir wollen aber hier möglichst "pythonisch" vorgehen. Zunächst zeigen wir, was das oben genannte Modell nicht so gut leistet. Hier zunächst eine einfache Klasse, wie oben besprochen.

In [2]:
class Myclass:
    def __init__(self, number):
        self.__number = number

    def get_number(self):
        return self.__number

    def set_number(self, number):
        self.__number = number
        

Wir haben das nur ein privates Attribut ```__number``` und den öffentlichen getter und setter. Wollen wir damit arbeiten, sieht das so aus.

In [3]:
n1 = Myclass(17)
n2 = Myclass(19)
print(n1.get_number())
#n2.number # das gibt es  nicht und
#n2.__number # das geht auch nicht (mit der trickreichen Fehlermeldung)

17


Wollen wir jetzt die 2 Zahlen, die in den Instanzen n1 und n2 gespeichert sind, addieren und das Ergebnis in ```n1.__number```  abspeichern, sieht das so aus: 

In [4]:
n1.set_number(n1.get_number()+n2.get_number())
print(n1.get_number())

36


Dies sieht nicht sehr schön aus! Der User würde sich wünschen, einfach etwas schreiben zu können wie:<br><br>
$n1.number = n1.number+n2.number$

Es wäre schön, eine solch einfache User-Schnittstelle zu erzeugen, aber wir wollen unser Schutzkonzept nicht einfach über Bord werfen. Gerne würden wir auch an unseren settern und gettern festhalten, weil diese ja z.B. zulassen, vor Setzen des Attributs die Usereingaben zu überprüfen oder Attribute nur an berechtigte User herauszugeben. Python bietet hierfür eine elegante Lösung in Form von Properties an. Properties lassen zu, daß wir unsere Attribute <b>öffentlich</b> machen, unsere getter und setter trotzdem verwenden und damit die Attribute trotzdem schützen. Hier ein Beispiel, wie das geht. Unser ```number``` Attribut wird geprüft, ob es kleiner 0 oder größer als 100 ist, dann wird es entsprechend auf 0 oder 100 begrenzt. Außerdem haben wir die Möglichkeit, einen Passwort-Check (nicht Praxistauglich!) einzubauen. Dieses wird bei jedem Lesezugriff abgefragt. 

In [7]:
class Myclass:

    def __init__(self, number):
        self.set_x(number) #Achtung wir rufen hier die Funktion set_x auf, wo die Zuweisung von x auf self.__x erfolgt.

    def get_the_number(self): #Name ist frei
        #hier Code zum Manipulieren des Zugriffs
        password=input("Passwort bitte zum Auslesen des Attributs:")
        if password!="passw":
            print("Falsches Passwort, Zugriff verweigert")
            return "Falsche Eingabe"
        return self.__number

    def set_x(self, number):
        if number < 0:
            self.__number = 0
        elif number > 100:
            self.__number = 100
        else:
            self.__number = number

    number = property(get_the_number, set_x) #erstes Argument getter, zweites setter
    #Name der getter und setter Funktionen get_my_att und set_x unabhängig vom Namen des Attributs (x)

Was haben wir Neues gemacht? Die Zeile ```number = property(get_the_number, set_x)``` erzeugt eine property für das Attribut number. Beim Verwenden von number außerhalb der Klasse wird beim Lesen auf die Methode ```get_the_number``` und beim Schreiben auf ```set_x``` umgeleitet. Dies sind die beiden Parameter der Property. 
Verwenden wir jetzt "number" außerhalb der Klasse, ist immer die <b>Property "number"</b> angesprochen. Wir zeigen hier, wie es funktioniert.


In [10]:
inst1=Myclass(4) #wir initialisieren __number auf 4 mit der Methode set_x (s.__init__())
inst1_nr=inst1.number #number ist die Property number!!!
                      # beim lesen also Umleitung auf get_the_number
print(f"inst1.number ist: {inst1_nr}") 
                                       
inst2=Myclass(999) #Achtung wird 100 wegen Abfrage in set_x
inst2_nr=inst2.number
print(f"inst2.number ist: {inst2_nr}") 
if inst1_nr != "Falsche Eingabe" and inst2_nr!= "Falsche Eingabe":
    print(f"Summe inst2.number+inst1.number ist: {inst2_nr+inst1_nr}")
    inst1.number=inst2_nr+inst1_nr
    
    print(f"inst1.number wurde jetzt verändert auf {inst1.number} nicht  auf 104 !!")
    


Passwort bitte zum Auslesen des Attributs:passw
inst1.number ist: 4
Passwort bitte zum Auslesen des Attributs:passw
inst2.number ist: 100
Summe inst2.number+inst1.number ist: 104
Passwort bitte zum Auslesen des Attributs:passw
inst1.number wurde jetzt verändert auf: 100 nicht  auf 104 !!


Dies ist genau die einfache Umsetzung des Interfaces für den User, die wir uns gewünscht haben. Wenn wir ```number``` verwenden, wird immer auf die getter und setter für ```__number``` umgeleitet. Dies Attribut ist privat und von außerhalb der Klasse nicht zu erreichen. Wir könnten genauso gut einen völlig anderen Namen dafür nehmen wie z.B. ```__x```. Für den User zählt nur der Name der Property. Auch die getter und setter Methoden können natürlich beliebig heißen. Wir können dadurch auch erzwingen, daß immer die Zugangsmethoden verwendet werden, die hier z.B. eine Begrenzung für ```__number``` enthalten.<br>
Ein Riesenfortschritt, ein Problem bleibt aber noch. Wenn der Benutzer der Klasse richtig rät. könnte er dieses versuchen:<br>


In [11]:
inst1.set_x(17)
print(inst1.number)

Passwort bitte zum Auslesen des Attributs:passw
17


Dadurch wurde jetzt ```__number``` überschrieben, was wir nicht wollen. Was tun? Machen wir einfach unsere Zugangsmethoden ebenfalls privat und diese Schwachstelle ist beseitigt. Im Klassendirectory erscheint als Eintrag übrigens der Name der Property, nicht also ```__number``` (s.u.)

In [14]:
class Myclass:

    def __init__(self, number):
        self.__set_number(number) #Achtung die Funktion heißt ja jetzt __set_number, nicht vergessen!

    def __get_the_number(self): 
        #hier Code zum Manipulieren des Zugriffs. z.B. "Geben Sie Passwort ein"
        return self.__number

    def __set_number(self, number):        
        if number < 0:
            self.__number = 0
        elif number > 100:
            self.__number = 100
        else:
            self.__number = number

    number = property(__get_the_number, __set_number) #erstes Argument getter, zweites setter
    
print(dir(Myclass))   
inst1=Myclass(4) #wir initialisieren __number auf 4 mit der Methode set_number (s.__init__())
print(f"inst1.__number ist: {inst1.number}") # number ist die Property number, wir wollen lesen,
                                            # also Umleitung auf get_the_number
inst2=Myclass(999) #Achtung wird 100
print(f"Summe inst2.__number+inst1.__number ist: {inst2.number+inst1.number}, denn inst2.__number ist 100!!")
inst1.number=inst2.number+inst1.number
print(f"inst1.__number wurde jetzt verändert auf: {inst1.number}, denn der Schreib-Zugriff geht über __set_number")

['_Myclass__get_the_number', '_Myclass__set_number', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'number']
inst1.__number ist: 4
Summe inst2.__number+inst1.__number ist: 104, denn inst2.__number ist 100!!
inst1.__number wurde jetzt verändert auf: 100, denn der Schreib-Zugriff geht über __set_number


In [ ]:
########Alles was ich hier versuche, ist nun geblockt:#############
#print(inst1.__number)
#inst1.__set_number(3)
#print(inst1.__get_number())
    

Was aber keinen Fehler anzeigt, und wo man denken könnte, daß eine Schwachstelle vorliegt, ist die zweite Zeile in der nächsten Zelle. Direkter Schreibzugriff auf ein privates Attribut???
Dies wird <b>so aber nicht ausgeführt</b>, es wird unabhängig von unserem geschützen Attribut ```__number``` ein weiteres Attribut angelegt mit demselben Namen, was public ist. Wir bekommen so keinen Zugang auf das private Attribut ```__number``` in unserer Klasse. 

In [15]:
inst1.number=17 #der setter läuft
inst1.__number=99 #dynamisch anlegen von __number, aber nicht das __number der Klasse!!!! Ist wie unten.
inst1.__bla=104 #dynamisch anlegen von __bla.
print(f"in der Klasse ist inst1.__number: {inst1.number}") #der getter läuft
print(f" im Klassendirectory erscheint nur der Name der property:\n{dir(Myclass)}") #im Klassendirectory findet sich nur number!! Nicht__number und auch nicht __bla.
print(f"das on the fly gesetzte neue Attribut __bla ist public zugänglich: {inst1.__bla}") #normaler Zugang auf neues Attribut
print(f"das on the fly gesetzte neue Attribut __number ist ebenfalls public zugänglich! {inst1.__number} \n hat aber nichts mit unserem Klassen-internen __number {inst1.number} zu tun!")
print(inst1.number) #der getter läuft

in der Klasse ist inst1.__number: 17
 im Klassendirectory erscheint nur der Name der property:
['_Myclass__get_the_number', '_Myclass__set_number', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'number']
das on the fly gesetzte neue Attribut __bla ist public zugänglich: 104
das on the fly gesetzte neue Attribut __number ist ebenfalls public zugänglich! 99 
 hat aber nichts mit unserem Klassen-internen __number 17 zu tun!
17


Dies ist also ein guter Mechanismus, die Klasse zu schützen. Müssen wir nun für alle Attribute eine Property anlegen und die Attribute und ihre Zugriffsmethoden privat machen? Das hängt natürlich davon ab, was die Wertigkeit des Attributs ist. Interne Attribute, die für Berechnungen in der Klasse verwendet werden und nur in der ```__init__``` gesetzt werden, und für die nie vorgesehen ist, daß sie vom User gelesen oder beschrieben werden, benötigen z.B. keine Property. Attribute, die beliebige Werte annehmen können, also beim Beschreiben nicht überprüft werden müssen und die jeder lesen darf, natürlich ebenfalls nicht. Unnötige Properties machen unsere Klassen nur unübersichtlich!

Daneben müssen wir uns darüber klar sein, daß unsere Schutzmechanismen in der Praxis eher bedeuten:<br><b>"Programmiererkollege, nutze meine privaten Attribute nur über das entsprechende Interface und die geschützten Attribute mit großer Vorsicht (und eventuell Rücksprache)!"</b><br><br>
Denn vor wem schützen wir unsere Klassen?<br><br>
Andere Programmierer im Team? Sie haben den Source-Code und könnten ihn theoretisch nach Belieben verändern.<br>
Programmierer anderer Teams? Sie benötigen zum Arbeiten meist ebenfalls den Source-Code (s.o.).<br>
Kunden? Sie bezahlen für das Programmieren und ihnen gehört normalerweise der Code.<br>
<b>Unsere Schutzkonzepte funktionieren nur, wenn wir die Zusammenarbeit der verschiedenen Programmierer streng regeln.<br>
    Kunden dürfen unsere Programme nur über die Schnittstellen benutzen, oder verlieren bei Änderungen die Gewährleistungsansprüche.


Wenn wir unsere Klassen wie oben besprochen schützen, sind wir absolut auf einer guten Grundlage. Es gibt aber noch eine Ergänzung, die wir ebenfalls besprechen wollen, ```Dekorateure```, mit denen wir unsere Zugriffsmethoden verändern. Zunächst allgemein die Vorstellung eines Dekorateurs, der zwar zum Kernpython gehört, aber nicht häufig verwendet wird. Die Schreibweise ist ```@Namen_des_Dekorateurs```. Was tut ein Dekorateur? Hier ein Beispiel:

In [16]:
def our_decorator(func): #Dekoratorfunktion
    def function_wrapper(x):
        print("Vor dem Aufruf von " + func.__name__)
        func(x)
        print("Nach dem Aufruf von " + func.__name__)
    return function_wrapper

@our_decorator #hier der Dekorateur
def foo(x):
    print("foo wurde mit " + str(x) + " aufgerufen!")
foo("first")
foo("second")


Vor dem Aufruf von foo
foo wurde mit first aufgerufen!
Nach dem Aufruf von foo
Vor dem Aufruf von foo
foo wurde mit second aufgerufen!
Nach dem Aufruf von foo


Der Dekorateur muß unmittelbar über der Funktion ```foo()```, die er dekoriert, stehen. <b>Bevor die zugehörige Funktion ```foo()``` abgearbeitet</b> wird, leitet er die Programmausführung auf die Dekoratorfunktion ```our_decorator()``` um, mit der aufrufenden Funktion ```foo()``` als Parameter (hier foo übergeben in func). Es wird von dieser Funktion die innere Funktion ```function_wrapper``` mit dem Parameter x der dekorierten Funktion ```foo()``` aufgerufen und deren Ergebnis returniert. Diese innere Funktion kennt automatisch alle Parameter der ```our_decorator()``` Funktion. (Diese Verschachtelung von Funktionen mit automatischer Parameterübertragung bezeichnet man als Closure.) Wir können also die dekorierte Funktion, ohne deren eigene Syntax zu verändern, mit der Dekoration anpassen. Hier ein Beispiel, das die unterschiedliche Ausführung einer dekorierten Funktion je nach Ergebnis einer Bedingung in der Dekoratorfunktion zeigt:

In [77]:
def dekorator(func):
    def innere_function(x):
        print(f"Input für innere Funktion ist: {x}")
        if x<0:
            return(-x)
        else:
            return(x) 
    return innere_function

@dekorator
def absolut(x):
    return 

print(absolut(-3))
print(absolut(3))

Input für innere Funktion ist: -3
3
Input für innere Funktion ist: 3
3


Ganz schön kompliziert, was hat das mit unseren Klassen zu tun? Wir zeigen nun ein Beispiel:

In [17]:
class Myclass:

    def __init__(self, num):
        self.number=num #sofortige Umleitung in den setter!!
    
    @property #markiert number() als getter für __number
    def number(self):             #muß number heißen
        #hier Code zum Manipulieren des Zugriffs. z.B. "Geben Sie Passwort ein"
        return self.__number
    
    @number.setter #markiert number() als setter für __number muß @number heißen!!
    def number(self, number):     #muß number heißen   
        if number < 0:
            self.__number = 0
        elif number > 100:
            self.__number = 100
        else:
            self.__number = number

x= Myclass(110)  
print(x.number)
    
 

100


Die Funktionalität entspricht zu 100% unserm vorherigen Beispiel. Was ist anders? In der ```__init___``` übertragen wir den Parameter ```num``` sofort in den Setter für ```__number```. Dieser Setter muß mit dem Dekorateur @Name_des_Attributs, also ```@number.setter``` dekoriert sein. Außerdem muß der Name der Setter-Funktion ebenfalls dem Attributnamen entsprechen. Für die Getter-Methode gilt die vom Bezeichner des Attributs unabhängige Dekorierung mit @property. Aber auch hier muß der Name der Methode dem Namen des Atributs entsprechen! Ist dies alles besser? Kürzer? Schöner? Vielleicht, man muß es gesehen haben, da viele Pythonistas diese Konstruktion verwenden, aber wie gesagt, auch mit der Property()-Syntax (wie weiter oben gezeigt) funktioniert alles perfekt. 

Zum Abschluß noch ein Beispiel zur Umwandlung von Brüchen in Dezimalzahlen mit mehreren Attributen, um die Syntax unmißverständlich zu zeigen. Alle Variablen sind so genannt, daß man sieht, was wo stehen muß. (Dies ist übrigens interessant, weil wir jeweils zwei Methoden mit dem gleichen Namen haben, aber mit unterschiedlicher Signatur bezüglich der Parameter. Das geht normalerweise natürlich nicht, aber die verschiedene Dekoration macht es möglich!!) Neben den beiden Properties haben wir noch gt_bruch(), die den Bruch in Dezimal verwandelt.

In [ ]:
from math import inf
class Myclass:

    def __init__(self, input_zähler,input_nenner):        
        self.zähler=input_zähler #sofortige Umleitung in den setter!!
        self.nenner=input_nenner
        
    @property #markiert number() als getter für __zähler
    def zähler(self): 
        #hier Code zum Manipulieren des Zugriffs. z.B. "Geben Sie Passwort ein"
        return self.__zähler
    
    @zähler.setter #markiert number() als setter für __zähler
    def zähler(self, in_zähler):
        self.__zähler = in_zähler
        
    @property #markiert nenner() als getter für __nenner
    def nenner(self): 
        return self.__nenner
    
    @nenner.setter #markiert number() als setter für __nenner
    def nenner(self, the_nenner):        
        if the_nenner == 0:
            print("Teilen durch 0 Fehler")
        self.__nenner=the_nenner
        
    def get_bruch(self):
        if self.__nenner==0:
            return "Keine Zahl, Teilen durch 0"
        else:
            return self.zähler/self.nenner

x= Myclass(11,20)  
print(f"Bruch {x.zähler}/{x.nenner} in dezimal ist: {x.get_bruch()} ")
y=Myclass(2,0)   
print(f"Bruch {y.zähler}/{y.nenner} in dezimal ist: {y.get_bruch()} ")

<img class="imgright" src="Images/drunk.jfif" width="200">

# Übung 1

Benutzen Sie die vorher erstellte Klasse "Betrunken", die einen "Randomwalk" abbildet. (Die Betrunkenen haben einen Standort mit x und y Koordinate (initial 0,0). Sie können sich schrittweise bewegen, zufällig um 1, -1 oder 0 in einem Schritt in jeweils beiden Koordinaten. Stellen Sie fest, wie weit der Betrunkene sich maximal von seinem Ausgangspunt 0,0 entfernt nach 10, 20, 50 Schritten.) Lassen Sie für jeden Versuch 10 Betrunkene gehen und bestimmen Sie als Ergebnis die maximal erreichte Entfernung. Bauen Sie die Klasse mit privaten Attributen auf mit einem getter und setter.<br>
<img class="imgright" src="Images/randomwalk.png"  width="600">

# Übung 2

Schreiben Sie die Klasse um, sodass die Betrunkenen nicht bei 0,0 anfangen sondern an beliebiger Stelle. Benutzen Sie properties und erlauben Sie, dass self.x und self.y von aussen manipuliert wird, aber nur durch Ablauf der privaten getter und setter Methoden.


# Übung 3

Schreiben Sie die Klasse um. Benutzen Sie Dekorateure für die setter und getter.

##### Wir wollen uns jetzt mit den magischen oder Dundermethoden beschäftigen, von denen wir mit ```__init``` und ```__del__``` ja schon 2 besprochen haben.